In [1]:
import gameEngine
import copy
import pickle
import concurrent.futures
from random import choice
from random import randint
from tqdm import tqdm

In [2]:
def run_single_game(p1_func,p2_func):
    s = gameEngine.gen_starting_state(2)
    latch = True
    p1_win = 0
    p2_win = 0
    while latch:
        action_list = gameEngine.gen_valid_moves(s)
        if len(action_list) == 0:
            s[1] = 'lose'
        else:
            if s[0] == 'p1':
                if p1_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p1_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            else:
                if p2_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p2_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            gameEngine.next_state(s,action_list,selected_index)
        if s[1] == 'win':
            if s[0] == 'p1':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
        elif s[1] == 'lose':
            if s[0] == 'p2':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
    return (p1_func,p2_func,p1_win,p2_win)

In [2]:
def run_single_game_save(p1_func,p2_func,game_id):
    s = gameEngine.gen_starting_state(2)
    s_hist = [copy.deepcopy(s)]
    latch = True
    p1_win = 0
    p2_win = 0
    heuristic_list = []
    selected_index = -1
    while latch:
        action_list = gameEngine.gen_valid_moves(s)
        if len(action_list) == 0:
            s[1] = 'lose'
        else:
            if s[0] == 'p1':
                if p1_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p1_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            else:
                if p2_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p2_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            s_temp = copy.deepcopy(s)
            s_temp.append(selected_index)
            s_temp.append(action_list)
            s_temp.append(heuristic_list)
            s_hist.append(s_temp)
            gameEngine.next_state(s,action_list,selected_index)
        if s[1] == 'win':
            if s[0] == 'p1':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
        elif s[1] == 'lose':
            if s[0] == 'p2':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
    filename = f"saved_state/game_state_{p1_func}_{p2_func}_{game_id}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(s_hist, file)
    return (p1_func,p2_func,p1_win,p2_win)

In [4]:
function_name = ['random','norm heuristic','god heuristic']
function_arr = [0,1,2]
game_max = 100
player_pairs = []
for p1 in range(len(function_arr)):
    for p2 in range(len(function_arr)):
        for i in range(game_max):
            player_pairs.append((p1,p2))

In [5]:
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = list(tqdm(executor.map(run_single_game, *zip(*player_pairs)), total=len(player_pairs)))

100%|██████████| 900/900 [00:00<00:00, 2838.40it/s]


In [3]:
function_name = ['random','norm heuristic','god heuristic']
function_arr = [0,1,2]
game_max = 100
player_pairs = []
for p1 in range(len(function_arr)):
    for p2 in range(len(function_arr)):
        for i in range(game_max):
            player_pairs.append((p1,p2, i))

In [4]:
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = list(tqdm(executor.map(run_single_game_save, *zip(*player_pairs)), total=len(player_pairs)))

100%|██████████| 900/900 [00:00<00:00, 1118.80it/s]
